In [ ]:
!pip install -q transformers torch pandas numpy scikit-learn tqdm matplotlib huggingface_hub peft

import os, subprocess, time

def stage_to_local(src_dir: str, dst_dir: str, marker_name=".staged_ok"):
    os.makedirs(dst_dir, exist_ok=True)
    marker = os.path.join(dst_dir, marker_name)

    # 이미 스테이징 완료면 스킵
    if os.path.exists(marker):
        print(f"[stage] already staged: {dst_dir}")
        return

    print(f"[stage] rsync from\n  {src_dir}\n  -> {dst_dir}")
    t0 = time.time()

    # rsync는 증분 복사라서 중간에 끊겨도 다시 실행하면 이어서 빨리 맞춰짐
    cmd = f"rsync -a --info=progress2 '{src_dir.rstrip('/')}/' '{dst_dir.rstrip('/')}/'"
    subprocess.run(["bash", "-lc", cmd], check=True)

    # 마커 생성
    with open(marker, "w") as f:
        f.write("ok\n")

    print(f"[stage] done in {time.time()-t0:.1f}s")

# ====== 0) 구글드라이브 마운트 & code 경로 설정 ======
import os, sys

# Colab이 아니면 drive mount가 실패할 수 있음(로컬/엘리스/기타 환경 호환)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception as e:
    print('[warn] drive mount skipped:', e)

# 프로젝트 루트(Drive 기준). 필요하면 여기만 바꾸면 됨.
DRIVE_ROOT = '/content/drive/MyDrive/TRAITHON'
DRIVE_CODE = os.path.join(DRIVE_ROOT, 'code')
DRIVE_ADAPTER = os.path.join(DRIVE_ROOT, 'models/adapter')

# /content 로컬 스테이징(빠름)
LOCAL_CODE       = '/content/code'
LOCAL_BASE_MODEL = '/content/A.X-4.0-Light'
LOCAL_ADAPTER    = '/content/ax4_adapter_final'

# (선택) Drive에 있는 모듈/어댑터를 /content로 스테이징 (rsync 기반, 증분 복사)
if os.path.exists(DRIVE_CODE):
    try:
        stage_to_local(DRIVE_CODE, LOCAL_CODE)
    except Exception as e:
        print('[warn] code staging failed:', e)

if os.path.exists(DRIVE_ADAPTER):
    try:
        stage_to_local(DRIVE_ADAPTER, LOCAL_ADAPTER)
    except Exception as e:
        print('[warn] adapter staging failed:', e)

# 실제로 import에 쓸 CODE_DIR 결정 (우선순위: /content/code -> Drive/code -> ./code)
if os.path.exists(LOCAL_CODE):
    CODE_DIR = LOCAL_CODE
elif os.path.exists(DRIVE_CODE):
    CODE_DIR = DRIVE_CODE
else:
    CODE_DIR = './code'

if CODE_DIR not in sys.path:
    sys.path.insert(0, CODE_DIR)

print('CODE_DIR =', CODE_DIR)
print('LOCAL_ADAPTER =', LOCAL_ADAPTER)

# 데이터셋 경로 추가 설정
DRIVE_DATA = os.path.join(DRIVE_ROOT, 'data/golden_set_v1_TL')
LOCAL_DATA = '/content/data'

if os.path.exists(DRIVE_DATA):
    try:
        # 데이터셋 5,000개를 /content/data로 복사 (이게 가장 중요!)
        stage_to_local(DRIVE_DATA, LOCAL_DATA)
    except Exception as e:
        print('[warn] data staging failed:', e)

# 이후 6번째 셀에서 DATA_PATH를 LOCAL_DATA로 바꿔서 사용하세요.
DATA_PATH = LOCAL_DATA
data_path = DATA_PATH

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[stage] already staged: /content/code
[stage] already staged: /content/ax4_adapter_final
CODE_DIR = /content/code
LOCAL_ADAPTER = /content/ax4_adapter_final
[stage] already staged: /content/data


In [ ]:
import pandas as pd
import json
import os
from datetime import datetime

# 1. 보고서 기반 임계값 및 가드레일 설정
CONFIG = {
    'EC': {'T': 3.558751, 'k_pre': 7},
    'ET': {'T': 3.909962, 'k_pre': 7},
    'GB': {'T': 3.436861, 'k_pre': 6},
    'IS': {'T': 3.527919, 'k_pre': 7},
    'LC': {'T': 4.224508, 'k_pre': 8},
    'PO': {'T': 3.693775, 'k_pre': 7},
    'SO': {'T': 3.673025, 'k_pre': 6}
}

def load_jsonl(path):
    data = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return pd.DataFrame(data)

def analyze_short_drift_fixed(data_path):
    print("[info] Loading and cleaning datasets...")
    ppl_df = load_jsonl(f"{data_path}/ppl_golden.jsonl")
    time_df = load_jsonl(f"{data_path}/golden_file_time_map.jsonl")

    # [교정] PPL 데이터의 rel_path에서 파일명만 추출하여 병합 키 생성
    ppl_df['basename'] = ppl_df['rel_path'].apply(lambda x: os.path.basename(x))

    # [교정] 파일명의 앞 2글자를 섹션으로 추출 (예: EC_... -> EC) [cite: 223]
    ppl_df['section'] = ppl_df['basename'].str[:2]

    # 데이터 병합 (basename 기준)
    df = pd.merge(ppl_df, time_df[['rel_path', 'dt_iso']], left_on='basename', right_on='rel_path', suffixes=('', '_time'))

    # 시간 데이터가 없는(None) 행 제외 및 날짜 변환 [cite: 226]
    df = df.dropna(subset=['dt_iso'])
    df['date'] = pd.to_datetime(df['dt_iso']).dt.date

    results = []
    for section, cfg in CONFIG.items():
        sec_df = df[df['section'] == section].copy()
        if sec_df.empty: continue

        T = cfg['T']
        k_pre = cfg['k_pre']

        # 기사별 exceed 판정 [cite: 182, 211]
        sec_df['exceed'] = (sec_df['mean_nll_deployed_on'] > T).astype(int)

        # 일 단위 집계 (X1) [cite: 184]
        daily = sec_df.groupby('date').agg(
            X1=('exceed', 'sum'),
            n1=('exceed', 'count'),
            avg_nll=('mean_nll_deployed_on', 'mean')
        ).reset_index().sort_values('date')

        # 7일 롤링 집계 (X7, n7) [cite: 185, 192]
        daily['X7'] = daily['X1'].rolling(window=7, min_periods=1).sum()
        daily['n7'] = daily['n1'].rolling(window=7, min_periods=1).sum()

        # 가드레일 상태 머신 적용 [cite: 186, 187, 188]
        status_list = []
        for i in range(len(daily)):
            curr = daily.iloc[i]
            prev = daily.iloc[i-1] if i > 0 else None

            # 표본수 부족 예외 처리 (n7 < 200) [cite: 192]
            if curr['n7'] < 200:
                status_list.append('INSUFFICIENT_DATA')
            # ALERT: WATCH 2일 연속 AND X1 >= 2 [cite: 188]
            elif prev is not None and prev['X7'] >= k_pre and curr['X7'] >= k_pre and curr['X1'] >= 2:
                status_list.append('ALERT')
            # WATCH: X7 >= k_pre [cite: 187]
            elif curr['X7'] >= k_pre:
                status_list.append('WATCH')
            else:
                status_list.append('NORMAL')

        daily['status'] = status_list
        daily['section'] = section
        results.append(daily)

    return pd.concat(results) if results else pd.DataFrame()

# 분석 실행
analysis_result = analyze_short_drift_fixed(DATA_PATH)

if not analysis_result.empty:
    print("\n" + "="*75)
    print(f"{'Section':<8} | {'Date':<12} | {'Avg_NLL':<8} | {'X7':<4} | {'n7':<4} | {'Status':<15}")
    print("-" * 75)
    # 각 섹션별 가장 최신 결과 출력
    latest = analysis_result.groupby('section').tail(1)
    for _, row in latest.iterrows():
        print(f"{row['section']:<8} | {str(row['date']):<12} | {row['avg_nll']:<8.4f} | {int(row['X7']):<4} | {int(row['n7']):<4} | {row['status']:<15}")
    print("="*75)

[info] Loading and cleaning datasets...

Section  | Date         | Avg_NLL  | X7   | n7   | Status         
---------------------------------------------------------------------------
EC       | 2022-07-29   | 2.5415   | 0    | 11   | INSUFFICIENT_DATA
ET       | 2022-07-24   | 3.2754   | 0    | 10   | INSUFFICIENT_DATA
GB       | 2022-07-17   | 2.1562   | 0    | 9    | INSUFFICIENT_DATA
IS       | 2022-07-27   | 2.6188   | 0    | 8    | INSUFFICIENT_DATA
LC       | 2022-02-25   | 3.5588   | 0    | 7    | INSUFFICIENT_DATA
PO       | 2022-08-03   | 2.6659   | 0    | 12   | INSUFFICIENT_DATA
SO       | 2022-07-20   | 2.8341   | 0    | 10   | INSUFFICIENT_DATA


In [ ]:
import json, os
import pandas as pd
from datetime import datetime, timedelta

# ====== 1) 경로 및 설정 세팅 (첫 번째 셀의 DATA_PATH 활용) ======
IN_PATH = os.path.join(DATA_PATH, "ppl_golden.jsonl")
OUT_DIR = "/content/out_short_drift"
TARGET_SECTION = "SO" # 시뮬레이션 대상 섹션
SEED = 42

# 보고서(Source 53, 62) 기반 임계값 및 가드레일 설정
T = {
    "EC": 3.558751, "ET": 3.909962, "GB": 3.436861,
    "IS": 3.527919, "LC": 4.224508, "PO": 3.693775, "SO": 3.673025
}
K_PRE = {"GB": 6, "SO": 6, "EC": 7, "ET": 7, "IS": 7, "PO": 7, "LC": 8}

os.makedirs(OUT_DIR, exist_ok=True)

# ====== 2) 데이터 로드 및 섹션 분리 ======
print(f"[info] Loading data from {IN_PATH}...")
rows = []
with open(IN_PATH, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            rows.append(json.loads(line))

df = pd.DataFrame(rows)
# newsID에서 섹션 코드 추출 (예: SO_M01... -> SO)
df["section"] = df["newsID"].str.split("_").str[0]

sec_df = df[df["section"] == TARGET_SECTION].copy()
thr = T[TARGET_SECTION]
k_pre = K_PRE[TARGET_SECTION]

# ====== 3) 슬라이스 생성 (정상 vs 위반) ======
# Baseline: 임계값보다 낮은 정상 기사 50개 샘플링 [cite: 55]
safe_pool = sec_df[sec_df["mean_nll_deployed_on"] <= thr].copy()
baseline = safe_pool.sample(n=50, random_state=SEED).copy()

# Violation: NLL 상위 50개 (드리프트 발생 상황 가정) [cite: 68, 71]
violation = sec_df.sort_values("mean_nll_deployed_on", ascending=False).head(50).copy()

def save_jsonl(df_, path):
    with open(path, "w", encoding="utf-8") as w:
        for _, r in df_.iterrows():
            w.write(json.dumps(r.to_dict(), ensure_ascii=False) + "\n")

save_jsonl(baseline, os.path.join(OUT_DIR, f"slice_{TARGET_SECTION}_baseline.jsonl"))
save_jsonl(violation, os.path.join(OUT_DIR, f"slice_{TARGET_SECTION}_violation.jsonl"))

# ====== 4) 9일간의 시나리오 시뮬레이션 ======
# 7일간은 정상 가동(n7 >= 200 조건 충족용), 이후 2일간 드리프트 발생 [cite: 64]
start_dt = datetime(2026, 1, 1)
daily_plan = [("BASELINE", baseline)] * 7 + [("VIOLATION", violation)] * 2

records = []
x1_hist = []
status = "CLEAR"
watch_streak = 0
alert_streak_clear = 0

print("[info] Running 9-day simulation...")
for day_idx, (tag, slice_df) in enumerate(daily_plan):
    day = start_dt + timedelta(days=day_idx)
    # 당일 임계값 초과 개수 X1 [cite: 56, 83]
    x1 = int((slice_df["mean_nll_deployed_on"] > thr).sum())
    x1_hist.append(x1)

    # 최근 7일간 초과 개수 합계 X7 [cite: 57, 84]
    last7 = x1_hist[-7:]
    n7 = 50 * len(last7)
    x7 = sum(last7)

    # 상태 머신 (Source 58-61 규칙 적용)
    if n7 < 200:
        cur = "INSUFFICIENT"
    else:
        if status != "ALERT":
            cur = "WATCH" if x7 >= k_pre else "CLEAR"
        else:
            cur = "ALERT"

        # WATCH 및 ALERT 진입 조건 판정 [cite: 60]
        if cur == "WATCH": watch_streak += 1
        else: watch_streak = 0

        if status != "ALERT" and watch_streak >= 2 and x1 >= 2:
            status = "ALERT"
            cur = "ALERT"

        # CLEAR 복구 조건 [cite: 61]
        if status == "ALERT":
            if x7 < k_pre: alert_streak_clear += 1
            else: alert_streak_clear = 0
            if alert_streak_clear >= 2:
                status = "CLEAR"
                cur = "CLEAR"
                watch_streak = 0

    records.append({
        "day": day.strftime("%Y-%m-%d"),
        "tag": tag, "X1": x1, "X7": x7, "n7": n7,
        "status": cur, "max_nll": float(slice_df["mean_nll_deployed_on"].max())
    })

# 결과 저장 및 출력
sim_df = pd.DataFrame(records)
out_csv = os.path.join(OUT_DIR, f"simulate_{TARGET_SECTION}_9days.csv")
sim_df.to_csv(out_csv, index=False, encoding="utf-8-sig")

print(f"\n[Success] Simulation CSV saved at: {out_csv}")
display(sim_df)

[info] Loading data from /content/data/ppl_golden.jsonl...
[info] Running 9-day simulation...

[Success] Simulation CSV saved at: /content/out_short_drift/simulate_SO_9days.csv


,day,tag,X1,X7,n7,status,max_nll
0,2026-01-01,BASELINE,0,0,50,INSUFFICIENT,3.605320
1,2026-01-02,BASELINE,0,0,100,INSUFFICIENT,3.605320
2,2026-01-03,BASELINE,0,0,150,INSUFFICIENT,3.605320
3,2026-01-04,BASELINE,0,0,200,CLEAR,3.605320
4,2026-01-05,BASELINE,0,0,250,CLEAR,3.605320
5,2026-01-06,BASELINE,0,0,300,CLEAR,3.605320
6,2026-01-07,BASELINE,0,0,350,CLEAR,3.605320
7,2026-01-08,VIOLATION,6,6,350,WATCH,4.200069
8,2026-01-09,VIOLATION,6,12,350,ALERT,4.200069
